# SAO/LIP Python Primer Course Exercise Set 12

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/acorreia61201/SAOPythonPrimer/blob/main/exercises/Exercises12.ipynb)

## Exercise: Writing an Integration Module

This exercise has been adapted from the original version of lecture 12. You'll be using the trapezoidal and Monte-Carlo methods of integration that we've been using throughout the exercises thus far.

1. Write a class called `Trapezoidal1D` that:
    a. Takes two argument upon initialization: a function and the number of samples to use.
    b. Defines an `integrate` method that takes as input the limits to integrate between and calculates the integral of the function using the [trapezoidal rule](https://en.wikipedia.org/wiki/Trapezoidal_rule).

For example, if you define the function:

```python
def f(x):
    return 1/(1+x*x)
```

Then you should be able initialize your class and integrate between -1 and 1 with:

```python
numerical_integrator = Trapezoidal1D(f, 100000)
numerical_integrator.integrate(-1, 1)
```

In [3]:
import numpy as np

class Trapezoidal1D:
    '''Evaluates an integral using the trapezoidal rule'''

    def __init__(self, f, n):
        '''Initialize function and n points'''
        self.f = f # define the function
        self.n = n # define the number of points

    def integrate(self, a, b):
        '''Integrate using trapezoidal rule on interval [a, b]'''
        grid = np.linspace(a, b, self.n) # grid with n points and bounds [a, b]
        dx = (b-a)/(self.n-1) # separation between points
        sum = self.f(a) + self.f(b) # placeholder containing endpoints
        for i in range(1, self.n-1): # iterate over all points except first and last
            sum += 2*self.f(grid[i]) # add each contribution
        return sum*dx/2 # multiply prefactor

# remember that we have to use self.variable if we refer to a variable defined in __init__

In [5]:
# testing with known values
def f(x):
    return x**2

sqint = Trapezoidal1D(f, 100000)
sqint.integrate(0, 1) # should get ~1/3

0.33333333335000087

In [6]:
def g(x):
    return np.exp(x)

expint = Trapezoidal1D(g, 100000)
expint.integrate(0, 1) # should be e - 1 ~ 1.718

1.7182818284733599

2. Write an abstract base class called `Integral1D` that defines a common API for all 1D integral classes. This class should:

 * Have the same `__init__` as the `Trapezoidal1D` you defined in (1).
 * Define an abstract method called `integrate` that takes in the limits.

In [8]:
from abc import ABC, abstractmethod

class Integral1D:
    '''Abstract base class for generalizing integrator API'''

    def __init__(self, f, n):
        '''Initialize function and n points'''
        self.f = f # define the function
        self.n = n # define the number of points

    @abstractmethod
    def integrate(self, a, b):
        '''Initializes the bounds [a, b] of integration'''
        pass # remember, this function doesn't actually do anything; we need a child class to inherit here

3. Modify `Trapezoidal1D` so that it inherits from `Integral1D`. You should be able to drop `Trapezoidal`'s `__init__` method since it will use the one in `Integral1D`.

In [9]:
# copying from above:
class Trapezoidal1D(Integral1D): # add Integral1D as an "argument" for inheritance
    '''Evaluates an integral using the trapezoidal rule'''

    # we don't need an __init__ here; Integral1D handles this for us

    def integrate(self, a, b):
        '''Integrate using trapezoidal rule on interval [a, b]'''
        grid = np.linspace(a, b, self.n) # grid with n points and bounds [a, b]
        dx = (b-a)/(self.n-1) # separation between points
        sum = self.f(a) + self.f(b) # placeholder containing endpoints
        for i in range(1, self.n-1): # iterate over all points except first and last
            sum += 2*self.f(grid[i]) # add each contribution
        return sum*dx/2 # multiply prefactor

In [10]:
# testing
sqint = Trapezoidal1D(f, 100000)
sqint.integrate(0, 1) # should get ~1/3

0.33333333335000087

4. Write a class called `MonteCarlo1D` that:
  * Inherits from `Integral1D`.
  * Implements [Monte Carlo integration](https://en.wikipedia.org/wiki/Monte_Carlo_integration) in it's `integral` method using the `N` samples instead of the trapezoidal rule.

In other words, using the function `f` above, you should be able to integrate between -1 and 1 with:

```python
numerical_integrator = MoneCarlo1D(f, 100000)
numerical_integrator.integrate(-1, 1)
```

In [13]:
class MonteCarlo1D(Integral1D): # inherit from Integral1D
    '''Integrate using the Monte Carlo method'''

    # again, we don't need an __init__ function; Integral1D handles it for us

    def integrate(self, a, b):
        '''Integrate using Monte-Carlo integration on interval [a, b]'''
        grid = np.random.uniform(a, b, self.n) # generate random grid
        V = b - a # 1D volume element
        sum = 0 # placeholder
        for i in grid:
            sum += self.f(i) # add each term
        return sum*V/self.n # multiply prefactor

In [17]:
# testing
sqint = MonteCarlo1D(f, 100000)
sqint.integrate(0, 1) # should get ~1/3

0.33403360996188886

In [18]:
expint = MonteCarlo1D(g, 100000)
expint.integrate(0, 1) # should be e - 1 ~ 1.718

1.7211881769787736